<a href="https://colab.research.google.com/github/bjpark-forest/2023-1-Intro_ML/blob/main/SVM_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
import requests
import io

# 임웅교수님 공유 링크에서 파일 ID 추출
file_url = "https://drive.google.com/file/d/1xKvl88xKXxMOEOoUWkTpXRoaElhN7RYo/view?usp=sharing"
file_id = file_url.split("/")[-2]

# 파일 다운로드
url = f"https://drive.google.com/uc?id={file_id}&export=download"
response = requests.get(url)
content = response.content

# CSV 파일 읽고 정보 확인
df = pd.read_csv(io.StringIO(content.decode('utf-8')))
df

In [ ]:
df.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
target = LE.fit_transform(df['diagnosis'])

df.drop('diagnosis', axis=1, inplace=True)
df['diagnosis'] = target
df

In [ ]:
X=df.drop(['diagnosis','id','Unnamed: 32'], axis=1)


In [ ]:
X

In [ ]:
y=df['diagnosis']
y

In [ ]:
sns.pairplot(df,hue='diagnosis',vars=['texture_mean','area_mean','radius_mean','compactness_mean','perimeter_mean','smoothness_mean'])

In [ ]:
X1=df[['perimeter_mean','smoothness_mean']]


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X2 = scaler.fit_transform(X1)
X2

In [ ]:
import random

from sklearn.model_selection import train_test_split
random.seed(10)
X_train, X_test, y_train, y_test = train_test_split(X2,y,test_size=0.3)

In [ ]:
from sklearn.svm import SVC

classifier = SVC(kernel='linear', C=1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

test_acc = accuracy_score(y_test, y_pred)
test_acc

In [ ]:
from sklearn.metrics import confusion_matrix
compare = confusion_matrix(y_test, y_pred)
compare

In [ ]:
accuracy = (compare[0][0]+compare[1][1])/compare.sum()
accuracy

In [ ]:
poly_svc = SVC(kernel="poly", C=1, coef0=1, degree=4)
poly_svc.fit(X_train, y_train)

y_pred = poly_svc.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

test_acc = accuracy_score(y_test, y_pred)
test_acc

In [ ]:
rbf_svc = SVC(kernel="rbf", gamma=10, C=1)
rbf_svc.fit(X_train, y_train)

y_pred = rbf_svc.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

test_acc = accuracy_score(y_test, y_pred)
test_acc

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = [
    {"kernel": ["linear"], "C": [0.1, 0.5, 1, 5, 10, 30]},
    {"kernel": ["poly"],"C": [0.1, 0.5, 1, 5, 10, 30] },
    {"kernel": ["rbf"], "C": [0.1, 0.5, 1, 5, 10, 30],
                      "gamma": [0.01, 0.03, 0.1, 0.3, 1.0, 3.0,10.0,50.0,100.0],},]

In [ ]:
grid_cv = GridSearchCV(classifier, params, cv=20,n_jobs=-1)
random.seed(123)
grid_cv.fit(X_train, y_train)


In [ ]:
print(f"Highest score of paramter search is: {grid_cv.best_score_:.4f}")

In [ ]:
print("The parameter of the highest score is as follows")
for key, value in grid_cv.best_params_.items():
    print(f"{key}: {value}")

In [ ]:
import numpy as np
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.02):
    markers = ('s', 'x')
    colors = ('red', 'blue')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution), np.arange(x2_min, x2_max, resolution))
    
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

# Define the classifiers
classifiers = [classifier, poly_svc, rbf_svc]
names = ['Linear SVC', 'Poly SVC', 'RBF SVC']

for clf, name in zip(classifiers, names):
    plt.figure()
    plot_decision_regions(X_train, y_train, classifier=clf)
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.title(name)
    plt.show()
